# MODNet on experimental dataset 

MODNet model with the fit_preset version, and doing joint learning on the experimental and PBE band gaps and their difference.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models import MODNetModel
from modnet.models.ensemble import EnsembleMODNetModel
import numpy as np
import os
import copy

from collections import defaultdict
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt 
from IPython.display import Markdown
from matminer.datasets import load_dataset, get_all_dataset_info
from pymatgen.core import Composition

from modnet.featurizers import MODFeaturizer
from modnet.featurizers.presets import DeBreuck2020Featurizer

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

In [3]:
md = MODData.load('exp_gap_all')
df = pd.read_pickle('df_exp_cleaned.pkl')
md_joint = MODData.load('exp_pbe_joint')

2021-05-05 11:59:12,513 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7ff7d0267e80> object, created with modnet version <=0.1.7
2021-05-05 11:59:13,401 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7ff626e78730> object, created with modnet version <=0.1.7


In [4]:
md_joint.df_targets

,exp_gap,pbe_gap,difference
mp-12699,0.00,0.0000,0.0000
mp-559459,3.40,2.1704,1.2296
mp-21162,0.00,0.0000,0.0000
mp-1306,0.00,0.0000,0.0000
mp-15252,0.53,0.1049,0.4251
...,...,...,...
mp-556541,1.80,1.5135,0.2865
mp-2602,1.04,0.0000,1.0400
mp-30366,0.00,0.0000,0.0000
mp-3718,1.45,0.2418,1.2082


In [5]:
import pymatgen.ext.matproj

In [6]:
from pymatgen.ext.matproj import MPRester
import pandas as pd

with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
    index = md_joint.df_structure.index
    array = np.empty(shape=md_joint.df_structure.shape, dtype=pymatgen.core.structure.Structure)
    array = np.ndarray.flatten(array)
    for i in range(len(md_joint.df_structure)):
        array[i] = mpr.get_structure_by_material_id(index[i])
    md_joint.df_structure = pd.DataFrame(data=array, columns=["structure"])
    md_joint.df_structure = md_joint.df_structure.set_index(index)

In [7]:
#md_joint.save('exp-alone_pbe_joint')

2021-05-05 12:06:27,358 - modnet - INFO - Data successfully saved as exp-alone_pbe_joint!


In [5]:
md_joint = MODData.load('exp-alone_pbe_joint')

2021-05-06 09:25:21,297 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f60ec22e280> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_structure

,structure
id,
mp-12699,"[[ 0.13616363 -2.60631371 1.50340165] Ca, [-1..."
mp-559459,"[[2.14049282 6.43464803 1.76199846] Cs, [-0.70..."
mp-21162,"[[0. 0. 0.] Sr, [0. 2.5259875 2.5259875..."
mp-1306,"[[3.259129 1.88166243 1.15971025] Th, [ 3.25..."
mp-15252,"[[0.95531657 1.042239 0.86229681] Cu, [2.803..."
...,...
mp-556541,"[[0. 6.49363948 5.82657217] In, [ 0. ..."
mp-2602,"[[3.023318 3.023318 3.023318] Nd, [0. 0. 0.] As]"
mp-30366,"[[4.51774558 1.37529211 4.32659015] Ca, [1.109..."


In [7]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
uncertainties = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
      
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    model = EnsembleMODNetModel([[['exp_gap']]],{'exp_gap':1})
    model.fit_preset(train, verbose=0, n_jobs=20)
    
    pred, std = model.predict(test, return_unc=True)
    #pred = pred.drop(columns='pbe_gap')
    true = test.df_targets
    error = pred['exp_gap']-true['exp_gap']
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    std = std['exp_gap'].drop(std.index[((std['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error).mean()
    uncertainty = np.abs(std).mean()
    print('mae')
    print(mae)
    print('uncertainty')
    print(uncertainty)
    maes[i] = mae
    uncertainties[i] = uncertainty
    

2021-05-06 09:25:22,644 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:22,658 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:22,666 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:22,680 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:22,688 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:22,701 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:22,710 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:22,724 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:22,732 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:22,746 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-05-06 09:25:23,113 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f6077e40400> object, created with modnet version 0.1.9
2021-05-06 09

  0%|          | 0/360 [00:00<?, ?it/s]

2021-05-06 09:28:24,483 - modnet - INFO - Preset #4 fitting finished, loss: 0.5405220150947571


  0%|          | 1/360 [02:57<17:44:15, 177.87s/it]

2021-05-06 09:28:32,890 - modnet - INFO - Preset #5 fitting finished, loss: 0.47943260073661803


...

2021-05-06 17:20:26,772 - modnet - INFO - Preset #55 fitting finished, loss: 0.44250184297561646


100%|█████████▉| 359/360 [1:29:18<00:14, 14.84s/it]

2021-05-06 17:22:22,910 - modnet - INFO - Preset #57 fitting finished, loss: 0.5677945256233216


100%|██████████| 360/360 [1:31:14<00:00, 15.21s/it]


2021-05-06 17:22:26,187 - modnet - INFO - Preset #33 resulted in lowest validation loss with params {'train_data': <modnet.preprocessing.MODData object at 0x7f6039c788b0>, 'targets': [[['exp_gap']]], 'weights': {'exp_gap': 1}, 'n_models': 5, 'num_classes': {'exp_gap': 0}, 'n_feat': 128, 'num_neurons': [[64], [32], [16], [16]], 'lr': 0.005, 'batch_size': 64, 'epochs': 1000, 'loss': 'mae', 'act': 'elu', 'out_act': 'linear', 'callbacks': [<tensorflow.python.keras.callbacks.EarlyStopping object at 0x7f601efc0c10>], 'preset_id': 32, 'fold_id': 2, 'verbose': 0, 'val_data': <modnet.preprocessing.MODData object at 0x7f605b797af0>}
mae
0.3610488800198381
uncertainty
0.28794017


In [8]:
maes

array([0.37161024, 0.37979697, 0.29356389, 0.35051085, 0.36104888])

In [9]:
maes.mean()

0.3513061661858493

In [10]:
uncertainties

array([0.34915224, 0.33423334, 0.2791425 , 0.30915102, 0.28794017])

In [11]:
uncertainties.mean()

0.3119238555431366

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: -1%